In [3]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer

In [4]:
# Configurar la GPU si está disponible
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

In [6]:
# Descargar el dataset
path_to_file = tf.keras.utils.get_file(
    "shakespeare.txt",
    "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt"
)

# Leer el contenido
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f"Longitud del texto: {len(text)} caracteres")


Longitud del texto: 1115394 caracteres


In [7]:
# Mostrar las primeras líneas
print(text[:500])


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [11]:
vocab = sorted(set(text))
print(f'{len(vocab)} caracteres únicos en el texto')


39 caracteres únicos en el texto


**MODELO CARACTER A CARACTER**

Crear capas de mapeo de caracteres a IDs y viceversa. La capa tf.keras.layers.StringLookup nos permite convertir cada caracter en un ID numerico.

In [15]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(sorted(set(text))), mask_token=None)

chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))



Generar secuencias de entrenamiento

In [16]:
seq_length = 100
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)



DEFINICION DEL MODELO RNN CON GRU

In [17]:
class CharacterModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(
            rnn_units,
            return_sequences=True,
            return_state=True,
            recurrent_initializer='glorot_uniform'
        )
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False):
        x = self.embedding(inputs)  # Aplicamos el embedding
        if states is None:
            states = tf.zeros((inputs.shape[0], self.gru.units))  # Generar estado inicial con tamaño correcto
        x, states = self.gru(x, initial_state=states)  # Pasamos estado inicial correcto
        x = self.dense(x)
        if return_state:
            return x, states
        else:
            return x


In [18]:
vocab_size = len(ids_from_chars.get_vocabulary())
embedding_dim = 256
rnn_units = 1024

model_char = CharacterModel(vocab_size, embedding_dim, rnn_units)
model_char.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True))


In [24]:
EPOCHS = 10
history = model_char.fit(dataset, epochs=EPOCHS)


Epoch 1/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 926s 5s/step - loss: 2.8926
Epoch 2/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 972s 5s/step - loss: 1.8444
Epoch 3/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 928s 5s/step - loss: 1.5629
Epoch 4/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 910s 5s/step - loss: 1.4319
Epoch 5/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 931s 5s/step - loss: 1.3572
Epoch 6/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 914s 5s/step - loss: 1.3026
Epoch 7/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 923s 5s/step - loss: 1.2550
Epoch 8/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 926s 5s/step - loss: 1.2078
Epoch 9/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 925s 5s/step - loss: 1.1705
Epoch 10/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 932s 5s/step - loss: 1.1262


**Generación de Texto**

In [25]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

    @tf.function
    def generate_one_step(self, inputs, states=None):
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()
        predicted_logits, states = self.model(inputs=input_ids, states=states, return_state=True)
        predicted_logits = predicted_logits[:, -1, :] / self.temperature
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)
        predicted_chars = self.chars_from_ids(predicted_ids)
        return predicted_chars, states


In [26]:
generator_char = OneStep(model_char, chars_from_ids, ids_from_chars)
states = None
next_char = tf.constant(["To be or not to be"])
result = [next_char]

for n in range(500):
    next_char, states = generator_char.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
print(result[0].numpy().decode('utf-8'))


To be or not to be private friend.

isabella:
o, that thou rightsing your days like men of that;
for might be two never half.

gloucester:
what very thought offer'd to my daughter,
and thus scope: and she'll say how learn, sir; 'puce,
of any that with in exerticely?

gloucester:

buckingham:
what, have i offenced wedcheard, green.

lady capulet:
ah, what my bodies life; and thush a mortal semm:
yet behtlembing than my imparised are here. if my lord,
what was how edward, and clifford,
dispetion'd crast in a hillfi


**MODELO PALABRA A PALABRA**

In [8]:


tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
vocab_size_word = len(tokenizer.word_index) + 1

sequences_word = tokenizer.texts_to_sequences([text])[0]


In [9]:
seq_length = 20
sequences = [
    sequences_word[i:i + seq_length + 1] for i in range(len(sequences_word) - seq_length)
]
sequences = np.array(sequences)


In [10]:
X, y = sequences[:, :-1], sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size_word)


In [19]:


model_word = Sequential([
    Embedding(vocab_size_word, embedding_dim, input_length=seq_length),
    LSTM(150, return_sequences=True),
    LSTM(150),
    Dense(vocab_size_word, activation='softmax')
])

model_word.compile(loss='categorical_crossentropy', optimizer='adam')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
EPOCHS = 10
model_word.fit(X, y, batch_size=64, epochs=EPOCHS)


In [ ]:
def generate_text_word(model, tokenizer, seed_text, num_words, temperature=1.0):
    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = tf.expand_dims(token_list[-seq_length:], axis=0)
        predictions = model.predict(token_list, verbose=0)[0]
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(
            tf.math.log(predictions), num_samples=1).numpy()[0, 0]

        output_word = tokenizer.index_word.get(predicted_id, "")
        seed_text += " " + output_word
    return seed_text


In [ ]:
seed_text = "To be or not to be"
print(generate_text_word(model_word, tokenizer, seed_text, num_words=50, temperature=1.0))
